In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from PIL import Image
import numpy as np
from datasets import load_image_data 
from models import ImageCaptioningModel, TextGenerationModel  
from utils import save_model, load_model 
from eval import evaluate_captioning  

class InteractiveImageCaptioner:
    def __init__(self, 
                 caption_model_name="Salesforce/blip-image-captioning-base",
                 text_gen_model_name="gpt2"):
        """
        Initialize image captioning and text generation models

        Args:
            caption_model_name (str): Hugging Face model for initial image captioning
            text_gen_model_name (str): Hugging Face model for text modification
        """
        # Image Captioning Model
        self.caption_model = ImageCaptioningModel(caption_model_name)

        # Text Generation Model
        self.tokenizer = AutoTokenizer.from_pretrained(text_gen_model_name)
        self.text_generator = AutoModelForCausalLM.from_pretrained(text_gen_model_name)
    
    def generate_initial_caption(self, image_path):
        """
        Generate initial caption for the image

        Args:
            image_path (str): Path to the image file

        Returns:
            str: Initial image caption
        """
        try:
            # Load and preprocess the image using dataset handling function
            image_data = load_image_data(image_path)
            
            # Generate caption using the caption model
            caption = self.caption_model.generate_caption(image_data)
            
            # Return the caption
            return caption if caption else "No caption generated"
        
        except Exception as e:
            print(f"Error generating initial caption: {e}")
            return "Caption generation failed"
    
    def modify_caption_with_emotion(self, initial_caption, user_emotion):
        """
        Modify caption based on user's emotional input

        Args:
            initial_caption (str): Original image caption
            user_emotion (str): User's desired emotion or description

        Returns:
            str: Modified caption
        """
        try:
            # Construct prompt for text generation
            prompt = f"The image shows {initial_caption}. Describe it with a {user_emotion} tone: "
            
            # Encode the prompt
            input_ids = self.tokenizer.encode(prompt, return_tensors='pt')
            
            # Generate modified text
            output = self.text_generator.generate(
                input_ids, 
                max_length=100, 
                num_return_sequences=1, 
                temperature=0.7
            )
            
            # Decode the generated text
            modified_caption = self.tokenizer.decode(output[0], skip_special_tokens=True)
            
            return modified_caption
        
        except Exception as e:
            print(f"Error modifying caption: {e}")
            return initial_caption

def main():
    # Path to your image (replace with actual path)
    IMAGE_PATH = '/Users/rohitharavindramyla/Desktop/CSCI2470_Project/coco2017/train2017/000000000009.jpg'
    
    # Initialize the interactive captioner
    captioner = InteractiveImageCaptioner()
    
    # Generate initial caption
    initial_caption = captioner.generate_initial_caption(IMAGE_PATH)
    print("Initial Caption:", initial_caption)
    
    # Interactive user input for emotion
    print("\n--- Interactive Image Captioning ---")
    print("Original Caption:", initial_caption)
    
    # Get user emotion input
    user_emotion = input("Enter an emotion or description to modify the caption (e.g., 'sad', 'excited', 'mysterious'): ")
    
    # Modify caption based on user input
    modified_caption = captioner.modify_caption_with_emotion(initial_caption, user_emotion)
    
    # Display results
    print("\n--- Final Results ---")
    print("Initial Caption:", initial_caption)
    print(f"Caption with {user_emotion} tone:", modified_caption)

if __name__ == "__main__":
    main()
